## Investigating the multiple channels on the oscilloscope data output

In [2]:
import os

In [ ]:
data_path = "/Users/hayden/Desktop/HRMT64 data"
if not os.path.exists(data_path):
    raise ValueError("Error: specified data_path doesn't exist")